# Bloque 2: Modelos predictivos

Melanie Oyarzún W.


**Resultado de aprendizaje esperado:**

Identificar datos de series temporales, sus particularidades y riesgos, en el contexto de posibles aplicaciones profesionales.

**Bibliografía recomendada:**

Stock & Watson, C.14 [link](https://www.dropbox.com/s/d09qcat340jy5q2/stock%20y%20watson%20-%20Introduccion%20a%20la%20Econometria.pdf?dl=0) ; Wooldridge, c.12 [link](https://www.dropbox.com/s/wrh0tuna27qp747/Wooldridge__Introductory_Econometrics_2nd_Ed_Solutions.pdf?dl=0), Gujarati, c.12  [link](https://www.dropbox.com/s/rrvth8vsa1c5gts/Gujarati-cap%2012.pdf?dl=0)

**Material de apoyo***

El taller cuenta con un repositorio  [https://github.com/melanieoyarzun/taller_IDS2021](https://github.com/melanieoyarzun/taller_IDS2021) en el cual está dispoinible todo el material.


In [ ]:
# Paquetes y settings

from dateutil.parser import parse 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import watermark


import sklearn
from sklearn.linear_model import LinearRegression

import scipy
from scipy import stats

#%matplotlib inline
%load_ext watermark

# setting de graficos

plt.figure(figsize=(5,3), dpi= 200, facecolor='w', edgecolor='k')



In [ ]:
%watermark -n -v -m -g -iv

# Supuestos no cumplidos y opciones


Para poder interpretar causalmente un modelo de regresión lineal multiple en series de tiempo necesitamos que se cumplan los siguientes supuestos de Gauss Markov:

1. Modelo lineal en parámetros
$$y_{t}=\beta_{0}+\beta_{1}x_{1t}+\dots+\beta_{1}x_{kt}+u_{t}$$

2. Media condicionada nula o exogeneidad (que ahora es estricta)
$$ E[u_{t}|X_{js}]=0 \quad \forall s$$
3. No hay colinealidad perfecta.
4.  Homoscedasticidad
$$ Var[u_{t}|X_{js}]=\sigma \quad \forall s $$

5 No hay correlación serial.
$$ Cov[u_{t}, u_{s}|X_{js}]=\sigma \quad \forall s\ne t$$

Cada uno de estos supuestos tiene diferentes implicancias:

1-3 es que el estimador de MCO $\hat{\beta}$ es insesgado.
4-5 que tiene minima varianza de los estimadores lineales (eficiente)

1-5 -> MELI

Vemos que hay dos elementos diferentes a corte transversal: cambia la exogeneidad a uno más estricto y aparece un nuevo supuesto, correlación serial. 


# Objetivos diferentes de los modelos

Los modelos de regresión se pueden usar con dos objetivos principales: predicción y explicación.  En este bloque nos enfocaremos en la primera aplicación.

Un modelo de regresión puede ser útil para la predicción, aún cuando ninguno de sus coeficientes tenga interpretación causal. 

Desde el punto de vista de la predicción, lo que es importante es que el modelo entregue una predicción lo más precisa posible. 

La idea base es aprovechar las peculiaridades de las series temporales: la dependencia temporal y tendencias, para identificar patrones en la data que enriquezcan la predicción aun cuando no tengan valor explicativo. Estos elementos los incluiremos DENTRO de la regresión, para que represente de mejor manera el fenómeno a predecir.


# Patrones de dependencia intertemporal

¿cuándo falla supuesto de exogeneidad estricta?

Cuando tenemos **particularidades temporales** actuando en el proceso estocástico:

* Efecto rezagado (variable independiente rezagada)
* Retroalimentación entre variables
* Auto-dependencia, la variable depende de si misma en el pasado (variable dependiente rezagada, AR(1) )



¿Cómo identificar?

## Estacionareidad

El análisis de 

Entenderemos estacionareidad si el futuro se parece al pasado, al menos en un sentido probabilistico.



Una serie es estacionaria si:

1. Su distribución de probabilidad no varia en el tiempo, 
2. 

### Ejemplo


Hay muchos ejemplos de tipos de no estacionareidad. Ilustremos los más tipicos con unos ejemplos ficticios.

In [ ]:
x = np.linspace(0, np.pi*10, 360)
y = np.sin(x)

Podemos generar los diferentes tipos con algunas manipulaciones algebráicas.

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 18))
axs[0][0].plot(x, y)
axs[0][0].set_title('Serie Estacionaria')
axs[0][0].set_xlabel('tiempo')
axs[0][0].set_ylabel('Amplitud')

axs[0][1].plot(x, y+x/10)
axs[0][1].set_title('Media cambiante')
axs[0][1].set_xlabel('time')
axs[0][1].set_ylabel('Amplitud')


axs[1][0].plot(x, y*x/10)
axs[1][0].set_title('Varianza cambiante')
axs[1][0].set_xlabel('time')
axs[1][0].set_ylabel('Amplitud')

axs[1][1].plot(np.sin(x+x*x/30))
axs[1][1].set_title('Co-variance canviante')
axs[1][1].set_xlabel('time')
axs[1][1].set_ylabel('Amplitud')

plt.tight_layout()

### Ausencia de estacionareidad 1: Tendencias

Un claro ejemplo de series no estacionarias es cuando hay tendencias. Podemos identificar la tendencia a tarvés de una **media movil**
Revisemoslo con los datos de pasajeros de aerolineas.

In [ ]:
# Cargar datos

airline = pd.read_csv('data/international-airline-passengers.csv', sep=';')
airline['Month'] = pd.to_datetime(airline['Month']+'-01')
airline.set_index('Month', inplace=True)

Con un grafico rápido, identificamos que etsá todo bien y que efectivamente se observa que la serie no es estacionaria.

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(airline.index, airline['Passengers'])
ax.set_xlabel('Año')
ax.set_ylabel('Pasajeros')

Pdemos identificar la tendencia en los datos, al calcular la media movil.

In [ ]:
def running_average(x, order):
    current = x[:order].sum()
    running = []
    
    for i in range(order, x.shape[0]):
        current += x[i]
        current -= x[i-order]
        running.append(current/order)
    
    return np.array(running)

Esta función es autoexplicativa. Simplemente corre en el dataset paso a paso y calcula la media en una ventana específica. Ahora podemos agregar esta línea de tendencia al gráfico anterior.

In [ ]:
trend = running_average(airline['Passengers'], 12)

fig, ax = plt.subplots(1, 1)
ax.plot(airline.index, airline['Passengers'])
ax.set_xlabel('Año')
ax.set_ylabel('Pasajeros')
ax.plot(airline.index[12:], trend, label='Tendencia')
ax.legend()

A la serie, entonces, le podemos sacar esta tendencia, al dividr.

In [ ]:
detrended = airline.iloc[12:].values.flatten()/trend

Y graficamente:

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(airline.index[12:], detrended)
ax.set_xlabel('Date')
ax.set_ylabel('Detrended value')

### Ausencia de estacionareidad 2: Estacionalidad (no confundir con estacionareidad)

Es cuando hay un patron claro d ciclos que se repite en el tiempo. Generalmente los identificamos a priori con una inspección del grafico. Podemos usar una función para identificarlos. 

In [ ]:
def plot_seasons(detrended, order, plot_mean = True):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    N = len(detrended)

    data = np.array([detrended[i::order] for i in range(order)])
    
    means = np.mean(data, axis=1)
    medians = np.median(data, axis=1)
    
    counts = [0]
    counts.extend([len(data[i]) for i in range(order)])
    counts = np.cumsum(counts)

    ticks = (counts[:-1]+counts[1]/2)
    
    for i in range(order):
        values = data[i, :]
        npoints = len(values)

        plt.plot(range(counts[i], counts[i+1]), values, c=colors[0])
        plt.plot(range(counts[i], counts[i+1]), np.ones(npoints)*means[i], c=colors[1])
        plt.plot(range(counts[i], counts[i+1]), np.ones(npoints)*medians[i], c=colors[2])

    plt.legend(['data', 'mean', 'median'])
    plt.xlabel('season')
    plt.ylabel('values')
    plt.xticks(ticks, np.arange(order));
    
    if plot_mean:
        plt.plot(ticks, means, c=colors[3])
    
    return means

Acá, simplemente vamos a graficar la curva para diferentes periodos en la temporada. Esto se hace yendo a traves del dataset con un stride igual al periodo estacional.

In [ ]:
means = plot_seasons(detrended, 12)

La figura tambien nos provee una forma archetípica de comportamiento estacional. COn esto, podememos terminar la descomposición de la data en tres componentes.

In [ ]:
# Descomposición multiplicativa
def decomposition(data, order, plot=True):
    values = data.values.flatten()
    trend = running_average(values, order)
    detrended = values[order:]/trend
    
    season = [detrended[i::order].mean() for i in range(order)]
    seasonality = np.array(season*(detrended.shape[0]//order+1))[:detrended.shape[0]]
    residuals = values[order:]/(trend*seasonality)

    if plot:
        fig, axs = plt.subplots(4, 1, figsize=(22, 16), sharex=True)
        index = data.index

        axs[0].plot(index, values)
        axs[0].set_title('Data original')
        
        axs[1].plot(index[order:], trend)
        axs[1].set_title('Tendencia')

        axs[2].plot(index[order:], detrended)
        axs[2].set_title('Estacionalidad')

        axs[3].plot(index[order:], residuals)
        axs[3].set_title('Residuos')
        
    return values, trend, seasonality, residuals

Con esto , el patron estacional se remueve al repetir el patron medio estacional identificado anteriormente y dividiendo, desde la data sin tendencia. El resultado de esta disvision son simpemente los residuos. Lo que nos muestra que esta descomposición es demasiado sencilla aun.

In [ ]:
values, trend, seasonality, residuals = decomposition(airline, 12)

Acá usamos una descomposición multiplicativa, pero este mismo proneso se podría haber hecho siguiendo una descomposición aditiva, con pequeños cambios al codigo.

### Ausencia de estacionareidad 3: Quiebre estructural

Otro tipo de no estacionaeridad se presenta cuando la función de regresión poblacional cambia en el transcurso de la las observaciones. Esto puede ocurrir por varios motivos, por ejemplo cambios en una politica económica, cambios en la estructura de la economía, un nuevo invento o disrupción tecnológica, etc.


Si ocurren tales "cambios estructurales" o "rupturas", entonces un modelo de regresión que no tenga en cuenta esos cambios puede proporcionar una base engañosa para la inferencia ya la predicción.

Las estrategias para identificar un cambio estructurale son varias revisaremos dos: 
- Contrastes de hipótesis comparando cambios en los coeficientes de regresión mediante estadístico F o Test de Chow. 
- La segunda es biuscar potenciales cambios estructurales desde la predicción: se simula que la mmuestra termina antes de lo que realmente lo hace y se comparan las predicciones. Los cambios estructurales se detectan cuando la capacidad de predicción es sustancialmente peor de lo esperado.

## Correlación serial y autocorrelación.


Los datos de series temporales, generalmente están relacionadas con sus valores pasados.

**Razones:** inercia, reacciones rezagadas, entre otras.

Metemáticamente:

(Autocovarianza)
$$ Cov[u_{t}, u_{s}|X_{js}] \neq \sigma \quad \forall s\ne t$$


* Cuando una variable depende de sus propios valores en el pasado se denomina **autocorrelacion**

* Si los valores de una variable X en el presente están correlacionados con valores pasados de otra variable, Y, se conoce como **correlación serial**

* A veces se usan los términos como sinónimos.

Podemos identificar la correlación serial de tres maneras:
- Graficamente en los rsiduos
- Estadísticamente mediante la prueba de WHite o de Breaush y Pagan.


### Realicemos un ejemplo ficticio

La mayoría de las medidas de correlación para series de tiempo están basadas en la correlación de Pearson, asi que partiremos definiendolo en una función.

In [ ]:
def pearson(x, y):
    meanx = x.mean()
    meany = y.mean()
    stdx = x.std()
    stdy = y.std()
    
    return np.mean((x - meanx) * (y - meany)) / (stdx * stdy)

In [ ]:
# Genermos una data aleatoria, que por ende no deberia tener quiebres ni tendencias.

x = np.random.random(1000)
y = np.random.random(1000)

Hay perfecta correlación propia

In [ ]:
pearson(x, x)


In [ ]:
pearson(y, y)

Por definición, dos sets de variables aleatorias estan no correlacionados. Veamos el grafico.

In [ ]:
fig, axs = plt.subplots(1, 1)
axs.plot(x, y, '*')
axs.set_xlabel('x')
axs.set_ylabel('y')

Por lo cual, el coeficiente de correlacón de Pearson debería ser muy cercano a cero.

In [ ]:
pearson(x,y)

Sin embargo, sin agregamos una tendencia a estos set de valores rapidamente tenemos una fuerte correlación **A PESAR** de que no estan relacionados en realidad.


In [ ]:
trend = np.linspace(1, 5, 1000)
pearson(x+trend, y+trend)

Por eeste motivo, **SIEMPRE** debemos analizar las condiciones de estacionareidad y tendencias antes de analizar una serie, y quitarle dichas tenednecia.s

### Auto-correlación

Como observamos, por construcción la correlación de una variable consigo misma es 1. Pero, si vamos aumentando un lag respecto a la observación, la correlación se difumina.

In [ ]:
def acf(x, lag=40):
    return np.array([1] + [pearson(x[:-i], x[i:]) for i in range(1, lag)])

In [ ]:
plt.bar(range(40), acf(x, 40))
plt.xlabel('lag')
plt.ylabel('ACF')

Entonces, podemos saber si hay autocorrelación si esta es significativamente diferente de cero. En nuestro ejemplo ficticio, podemos calcular el intervalo de confianza y descartar cualquiera que caiga dentro de este.

In [ ]:
def acf_ci(acfv, n, alpha=0.05):
    se = [1 / np.sqrt(n)]
    se.extend(np.sqrt((1+2*np.cumsum(np.power(acfv[1:-1], 2)))/n))
    se = np.array(se)
    
    se *= stats.norm.ppf(1-alpha/2.)
    return se

In [ ]:
def plot_acf(x, lag=40, alpha=0.05):
    acf_val = acf(x, lag)
    
    plt.vlines(range(lag), 0, acf_val)
    plt.scatter(np.arange(lag), acf_val, marker='o')
    plt.xlabel('lag')
    plt.ylabel('ACF')
    
    # Determine confidence interval
    ci = acf_ci(acf_val, len(x), alpha)
    plt.fill_between(np.arange(1, ci.shape[0] + 1), -ci, ci, alpha=0.25)

In [ ]:
plot_acf(x)

Pasemos a un ejemplo real, volvamos a la serie de GDP que trabajamos al principio:

In [ ]:
GDP = pd.read_csv('data/GDP.csv', parse_dates=['DATE'])
GDP.set_index('DATE', inplace=True)

In [ ]:
ax=GDP.plot(legend=False)
ax.set_ylabel(r'GDP ($\$B$)')

In [ ]:
values = GDP['GDP']
detrended = values[1:]-values[:-1]

Como la serie tiene una ferte tendencia, la auto-correlación parece ser significativa inluso para pargos periodos.

In [ ]:
plot_acf(values)

Esto solo nos dice, que con mayor razon debemos desestacionalizar la serie.

In [ ]:
plt.plot(GDP.index[1:], detrended)
plt.xlabel('DATE')
plt.ylabel(r'QoQ Change ($\$B$)')

In [ ]:
plot_acf(detrended)

AHora es mucho más informativa

### Auto-correlación parcial

LA función de autocorrelación considera la serie completa para cada lag

In [ ]:
def rolling(x, order):
    npoints = x.shape[0]
    running = []
    
    for i in range(npoints-order+1):
        running.append(x[i:i+order])
        
    return np.array(running)

Podemos definir silimar la correlación parcial, slo que en cada lag trata de tener en cuenta la cantidad de correlación que ya ha sido explicada por los anteriores lags usando un modelo lineal para predecir $x_t$ e $x_{t-1}$

In [ ]:
def pacf(x, lag=40):
    y = []
    
    for i in range(3, lag + 2):
        windows = rolling(x, i)

        xt = windows[:, -1] # Current values are at the end
        xt_l = windows[:, 0] # Lagged values are at 0
        inter = windows[:, 1:-1] # Intermediate values are in between 1 and -1
        
        
        lm = LinearRegression(fit_intercept=False).fit(inter, xt)
        xt_fit = lm.predict(inter)

        lm = LinearRegression(fit_intercept=False).fit(inter, xt_l)
        xt_l_fit = lm.predict(inter)

        y.append(pearson(xt - xt_fit, xt_l - xt_l_fit))
    
    # Pad the array with the two missing values
    pacf_1 = acf(x, 2)[1]
    return np.array([1, pacf_1] +  y)

In [ ]:
def plot_pacf(x, alpha=0.05, lag=40):
    pacf_val = pacf(x, lag)
    plt.vlines(np.arange(lag + 1), 0, pacf_val)
    plt.scatter(np.arange(lag + 1), pacf_val, marker='o')
    plt.xlabel('lag')
    plt.ylabel('PACF')
    
    # Determine confidence interval
    ci = acf_ci(pacf_val, len(x))
    plt.fill_between(np.arange(1, ci.shape[0] + 1), -ci, ci, alpha=0.25)

In [ ]:
plot_pacf(detrended)

# Modelos Autorregresivos

## MEDIAS MOVILES (MA)


Empezamos por definir un modelo de media movil. Este es un modelo autorregresivo de "memoria corta" que se define como:

$$ x_t=\epsilon_t + \theta \epsilon_{t-1} $$

In [ ]:
def MA(epsilon, theta, mean=0):
    theta = list(theta)
    N = len(epsilon)
    theta0 = np.array([1] + theta)
    theta0 = theta0[::-1] # Invert the order
    q = len(theta0)
    
    X = []
    for i in range(N-q):
        X.append(np.dot(theta0, epsilon[i:i+q])+mean)
    
    return np.array(X)

Esta función simplemente multiplica el coeficiente $\theta$ con las variables estocasticas. 
Generamos algunos ejemplos y los visualizamos.

In [ ]:
N = 500
thetas = []
X_MA = []
q_max = 8

epsilon = np.random.normal(size=N)

for q in range(1, q_max+1, 2):
    thetas.append(np.random.random(q))
    X_MA.append(MA(epsilon, thetas[-1]))

Y las visualizamos lado a lado.

In [ ]:
fig, axs = plt.subplots(4, 1)

for i in range(q_max//2):
    axs[i].plot(X_MA[i])
    axs[i].legend([r'$\theta=\left[%s\right]$' % ", ".join(thetas[i].round(2).astype('str'))], prop={'size': 14})
    axs[i].get_xaxis().set_visible(False)
    
axs[-1].get_xaxis().set_visible(True)
fig.tight_layout()

Ahora deseamos calcular el grafico de autocorrelación, asi que re introducimos las funciones que ya definimos

In [ ]:
def pearson(x, y):
    meanx = x.mean()
    meany = y.mean()
    stdx = x.std()
    stdy = y.std()
    return np.mean((x - meanx) * (y - meany)) / (stdx * stdy)

def acf(x, lag=40):
    return np.array([1] + [pearson(x[:-i], x[i:]) for i in range(1, lag)])

def acf_ci(acfv, n, alpha=0.05):
    se = [1 / np.sqrt(n)]
    se.extend(np.sqrt((1+2*np.cumsum(np.power(acfv[1:-1], 2)))/n))
    se = np.array(se)
    
    se *= stats.norm.ppf(1-alpha/2.)
    return se

def plot_acf(x, lag=40, alpha=0.05):
    acf_val = acf(x, lag)
    
    plt.vlines(range(lag), 0, acf_val)
    plt.scatter(np.arange(lag), acf_val, marker='o')
    plt.xlabel('lag')
    plt.ylabel('ACF')
    
    # Determine confidence interval
    ci = acf_ci(acf_val, len(x), alpha)
    plt.fill_between(np.arange(1, ci.shape[0] + 1), -ci, ci, alpha=0.25)

In [ ]:
plot_acf(X_MA[0])

In [ ]:
plot_acf(X_MA[-1])

Es observable que el número de lags significantes, pareciera ser elmismo que en el modelo MA.

## ARIMA

Ahora definimos el modelo autorregresivo. La función es similar a la MA pero usa posiciones en lugar de los valores estocásticos brutos
Su ecuación es:



In [ ]:
def AR(epsilon, phi, drift=0):
    phi = list(phi)
    N = len(epsilon)

    phi0 = np.array([1] + phi)
    phi0 = phi0[::-1] # Invert the order
    p = len(phi0)
    
    X = epsilon.copy()

    for i in range(p, N):
        X[i-1] = np.dot(X[i-p:i], phi0) + drift
        
    return np.array(X)

Generamos algunos ejemplos y los vizualizamos

In [ ]:
N = 500
phis = []
X_AR = []
p_max = 8

epsilon = np.random.normal(size=N)

for p in range(1, p_max+1, 2):
    phis.append(np.random.normal(0, .2, p))
    X_AR.append(AR(epsilon, phis[-1]))

In [ ]:
fig, axs = plt.subplots(4, 1)

for i in range(p_max//2):
    axs[i].plot(X_AR[i])
    axs[i].legend([r'$\phi=\left[%s\right]$' % ", ".join(phis[i].round(2).astype('str'))], prop={'size': 14})
    axs[i].get_xaxis().set_visible(False)
    
axs[-1].get_xaxis().set_visible(True)
fig.tight_layout()

Quizás quisiste decir: And introduce the required functions for the Partial Autocorrelation function

In [ ]:
def rolling(x, order):
    npoints = x.shape[0]
    running = []
    
    for i in range(npoints-order):
        running.append(x[i:i+order])
        
    return np.array(running)

In [ ]:
def pacf(x, lag=40):
    y = []
    
    for i in range(3, lag + 2):
        windows = rolling(x.flatten(), i)

        xt = windows[:, -1] # Current values are at the end
        xt_l = windows[:, 0] # Lagged values are at 0
        inter = windows[:, 1:-1] # Intermediate values are in between 1 and -1

        lm = LinearRegression(fit_intercept=False).fit(inter, xt)
        xt_fit = lm.predict(inter)

        lm = LinearRegression(fit_intercept=False).fit(inter, xt_l)
        xt_l_fit = lm.predict(inter)

        y.append(pearson(xt - xt_fit, xt_l - xt_l_fit))
    
    # Pad the array with the two missing values
    pacf_1 = acf(x, 2)[1]
    return np.array([1, pacf_1] +  y)

In [ ]:
def plot_pacf(x, alpha=0.05, lag=40):
    pacf_val = pacf(x, lag)
    plt.vlines(np.arange(lag + 1), 0, pacf_val)
    plt.scatter(np.arange(lag + 1), pacf_val, marker='o')
    plt.xlabel('lag')
    plt.ylabel('PACF')
    
    # Determine confidence interval
    ci = acf_ci(pacf_val, len(x))
    plt.fill_between(np.arange(1, ci.shape[0] + 1), -ci, ci, alpha=0.25)

In [ ]:
plot_pacf(X_AR[0])

In [ ]:
plot_pacf(X_AR[-2])

# ARIMA

FInalmente, la combinación de ambos modelamientos es 

In [ ]:
def differentiate(values, d=1):
    # First value is required so that we can recover the original values with np.cumsum
    x = np.concatenate([[values[0]], values[1:]-values[:-1]])

    if d == 1:
        return x
    else:    
        return differentiate(x, d - 1)

In [ ]:
def integrate(values, d=1):
    x = np.cumsum(values)
    
    if d == 1:
        return x
    else:
        
        return integrate(x, d-1)

La clase de ARIMA es una variante de la definida en https://www.ritchievink.com/blog/2018/09/26/algorithm-breakdown-ar-ma-and-arima-models/.

In [ ]:
class ARIMA(LinearRegression):
    def __init__(self, q, d, p):
        """
        An ARIMA model.
        :param q: (int) Order of the MA model.
        :param p: (int) Order of the AR model.
        :param d: (int) Number of times the data needs to be differenced.
        """
        super().__init__(True)
        self.p = p
        self.d = d
        self.q = q
        self.ar = None
        self.resid = None
        
    def prepare_features(self, x):
        if self.d > 0:
            x = differentiate(x, self.d)
                    
        ar_features = None
        ma_features = None
        
        # Determine the features and the epsilon terms for the MA process
        if self.q > 0:
            if self.ar is None:
                self.ar = ARIMA(0, 0, self.p)
                self.ar.fit_predict(x)
            eps = self.ar.resid
            eps[0] = 0
            
            # prepend with zeros as there are no residuals_t-k in the first X_t
            ma_features = rolling(np.r_[np.zeros(self.q), eps], self.q)
            
        # Determine the features for the AR process
        if self.p > 0:
            # prepend with zeros as there are no X_t-k in the first X_t
            ar_features = rolling(np.r_[np.zeros(self.p), x], self.p)
                                
        if ar_features is not None and ma_features is not None:
            n = min(len(ar_features), len(ma_features)) 
            ar_features = ar_features[:n]
            ma_features = ma_features[:n]
            features = np.hstack((ar_features, ma_features))
        elif ma_features is not None: 
            n = len(ma_features)
            features = ma_features[:n]
        else:
            n = len(ar_features)
            features = ar_features[:n]
        
        return features, x[:n]
    
    def fit(self, x):
        features, x = self.prepare_features(x)
        super().fit(features, x)
        return features
            
    def fit_predict(self, x): 
        """
        Fit and transform input
        :param x: (array) with time series.
        """
        features = self.fit(x)
        return self.predict(x, prepared=(features))
    
    def predict(self, x, **kwargs):
        """
        :param x: (array)
        :kwargs:
            prepared: (tpl) containing the features, eps and x
        """
        features = kwargs.get('prepared', None)
        if features is None:
            features, x = self.prepare_features(x)
        
        y = super().predict(features)
        self.resid = x - y

        return self.return_output(y)
    
    def return_output(self, x):
        if self.d > 0:
            x = integrate(x, self.d) 
        return x
    
    def forecast(self, x, n):
        """
        Forecast the time series.
        
        :param x: (array) Current time steps.
        :param n: (int) Number of time steps in the future.
        """
        features, x = self.prepare_features(x)
        y = super().predict(features)
        
        # Append n time steps as zeros. Because the epsilon terms are unknown
        y = np.r_[y, np.zeros(n)]
        for i in range(n):
            feat = np.r_[y[-(self.p + n) + i: -n + i], np.zeros(self.q)]
            y[x.shape[0] + i] = super().predict(feat[None, :])
        return self.return_output(y)

### Ejemplo

Analicemos un dataset real, colvamos a los datos de mortalidad por Influenza y Neumonia CDC

In [ ]:
ILI = pd.read_csv('data/CDC.csv')
ILI['date'] = ILI['Year']+ILI['Week']/52.

In [ ]:
values = ILI['Percent of Deaths Due to Pneumonia and Influenza'].values
diff = differentiate(values, 1)

Recordémoslo rápidamente:

In [ ]:
plt.plot(values)

Ahora calculemos la funcion ACF

In [ ]:
plt.plot(diff[1:])

In [ ]:
plot_acf(diff)

Lo que indica que p=2. AHora grafiquemos PACF

In [ ]:
plot_pacf(diff)

Y observamos tambien que w es 2, por lo cual nuestro modelo simplemente es:

In [ ]:
model = ARIMA(2, 0, 2)
pred = model.fit_predict(values)

Comparemos los valores ajustados con el dataset original

In [ ]:
ax = plt.subplot(111)
ax.plot(pred[1:], label='forecast', lw=2)
ax.plot(values, label='y', lw=2)
plt.legend()

Podemos observar que el ajuste es bastante bueno, a pesar de que es un enfoque muy simple.  Para comparar, tambien incluimos los resultados de un modelo más sofisticado.

In [ ]:
results = sm.tsa.ARIMA(values, (2, 0, 2)).fit()

fig, ax = plt.subplots(1, 1)
pred_sm = results.plot_predict(ax=ax)
ax.plot(pred[1:])
plt.legend(['statsmodels forecast', 'y', 'simple forecast'])

# Prueba estadística para estadística para tendencias estocásticas

Hasta ahora, nuestra implementación ha sino principalmente grafica. Para detectartendencias estodásticas se suele usar el contraste de **raiz unitaria** AR

Se usa la idea de que , al menos en muestras grandes, el coeficiente de autocorrelación  estará cerca de 1 si hay tendencia estocástica. Por lo cual se puede contrastar mediante una prueba de hipótesis de que hay tendencia estocástica, frente a la de que no hay ninguna tendencia.

Uno de los constrastes más comunes es el de **Dickey-Fuller**

EN AR(1), se estima el paseo aleatorio y se contrasta la hipótesis 

$$ \text{H}_0: \beta_1=1 vs \text{H}_0: \beta_1<1  \text{en} y_t=\beta_0+ \beta_1y_{t-1}+u_t$$

Una forma más facil de realizar este test, es transformando el modelo mediante la diferencia:

$$ \text{H}_0: \gama=0 vs \text{H}_0: \gamma<1  \text{en} \Delta y_t=\beta_0+ \gamma y_{t-1}+u_t$$


**Dickey-FUller Aumentado**

Cuando no sabemos que tipo de proceso autorregresivo tienen los datos, se puede generalizar el analisis usando el ADF. En este el modelo autorregresivo tiene p rezagos. 

In [ ]:
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
series = read_csv('data/international-airline-passengers.csv', header=0, index_col=0, squeeze=True)
X = series.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
from statsmodels.tsa.stattools import adfuller
from numpy import log
series = read_csv('data/international-airline-passengers.csv', header=0, index_col=0, squeeze=True)
X = series.values
X = log(X)
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# Eligiendo un modelo

Generalmente, la eleccion de modelos y la longitud de sus rezagos se hace mediante AIC o BIC.

AIC (Akaike Information Criterion)
 * Menor AIC indica un mejor modelo
  
BIC (Bayesian Information Criterion)
  * Muy similar a AIC
  * Lower BIC indicates a better model

AIC vs BIC
La diferencia entre ambos, es como penalizan la complejidad del modelo.
BIC favorece modelos más simples que AIC.
Generalmente, AIC se prefiere para modelos predictivos y BIC para explicativos.


Veamos un ejemplo:
Supongamos que tenemos un modelo ARIMA del GDP, pero no sabemos cuantos rezagos hay. QUeremos comparar el caso con  0, 1 o 2 rezagos.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import itertools


# Grid Search para explorar opciones
p = d = q = range(0,3) # p, d, and q can be either 0, 1, or 2

pdq = list(itertools.product(p,d,q)) # gets all possible combinations of p, d, and q
combs = {} # stores aic and order pairs
aics = [] # stores aics

# Grid Search continued
for combination in pdq:
    try:
        model = ARIMA(ILI, order=combination) # create all possible models
        model = model.fit()
        combs.update({model.aic : combination}) # store combinations
        aics.append(model.aic)
    except:
        continue
        
best_aic = min(aics)

# Model Creation and Forecasting
model = ARIMA(data, order=combs[best_aic])
model = model.fit()
model.forecast(7)[0]

# Bloque 2: Aplicaicón

Considere el mecanismo para extraer datos de la sección anterior. Elija un país a analozar y realice las siguientes operaciones.

1. Calule $y_t=ln(GDP_t)$, que es el logaritmo del PIB real y $\Delta y_t$ como la tasa de crecimiento anual del PIB para el periodo. 
2. Estime la media de $\Delta y_t$
3. Exprese la tasa media de crecimeinto en puntos porcentuales de variación anual.
4. Estme la desvicaón estandar de $\Delta y_t$
5. Estime las cuatro primeras autocorrelaciones de $\Delta y_t$, ¿cuales son las unidades de medidas de estas autocorrelaciones?
6. Estime un modelo AR(1) para $\Delta y_t$. ¿Cual es el coeficiente para el vector autorregresivo? ¿Es estadisticamente significativo? Construya un intervalo de confianza para el estimador poblacional.
7. Estime un modelo AR(2) para $\Delta y_t$. ¿Cual es el coeficiente para el vector autorregresivo? ¿Es estadisticamente significativo? Construya un intervalo de confianza para el estimador poblacional.
8. Estime un modelo AR(3) y AR(4) para $\Delta y_t$. Utilice el criterio BIC y AIC para elegir el numero de rezagos.
9. Utilice un estadístico de DIckey- FUlller aumentado para contrastar la presencia de raiz unitaria. COmo alternativa, suponga que Y es estacionaria al rededor de una detendecia determinístuca.
10. Identifique la tendencia y posible estacionalidad y haga la descomposción grafica.
11. Contraste la existencia de un cambio estructural para AR(1) usando QLR.